# Introduction

## What is KNN?

The K-Nearest Neighbors (KNN) algorithm is a simple, intuitive, and powerful machine learning technique used for classification and regression tasks. It belongs to the family of instance-based learning algorithms, also known as lazy learning algorithms, because it does not build an explicit model during the training phase. Instead, it memorizes the training dataset and makes predictions based on the most similar instances (neighbors) in the dataset.

## How Does KNN Work?

KNN operates based on the following steps:
1. **Storing the Dataset**: During the training phase, KNN simply stores the training data points along with their corresponding labels.
2. **Calculating Distance**: When a new data point needs to be classified, KNN calculates the distance between this new point and all the points in the training dataset. Common distance metrics include Euclidean distance, Manhattan distance, and Minkowski distance.
3. **Finding Neighbors**: It then identifies the 'k' nearest neighbors to the new data point. The value of 'k' is a hyperparameter that determines the number of closest neighbors to consider.
4. **Majority Voting**: For classification tasks, KNN assigns the most frequent label among the 'k' neighbors to the new data point. For regression tasks, it computes the average value of the 'k' neighbors.

## Strengths of KNN

- **Simplicity**: KNN is easy to understand and implement, making it an excellent choice for beginners in machine learning.
- **No Training Phase**: Since KNN is a lazy learning algorithm, there is no explicit training phase, which can save computational resources.
- **Flexibility**: KNN can be used for both classification and regression tasks.
- **Adaptability**: KNN can naturally handle multi-class classification problems.

## Weaknesses of KNN

- **Computational Cost**: The prediction phase can be slow, especially with large datasets, as it involves calculating the distance between the new data point and all training points.
- **Memory Usage**: KNN requires storing the entire training dataset, which can be memory-intensive.
- **Sensitivity to Irrelevant Features**: KNN can be adversely affected by irrelevant or redundant features, which can distort distance calculations.
- **Choice of 'k' and Distance Metric**: The performance of KNN heavily depends on the choice of 'k' and the distance metric used. An inappropriate choice can lead to poor performance.

![K Nearest Neighbors illustration by GeeksforGeeks](https://media.geeksforgeeks.org/wp-content/uploads/20231214111348/K-Nearest-Neighbors-(1)-660.png "Source: GeeksforGeeks")

## Enhancing KNN with Learnable Weights

In this notebook, I introduce an enhancement to the traditional KNN algorithm by incorporating feature weights that are learned using gradient descent. This approach aims to address some of the weaknesses of standard KNN, particularly its sensitivity to irrelevant features. By assigning and optimizing weights for each feature, our Dynamic Weighted KNN algorithm adapts to the importance of different features, potentially improving classification accuracy and robustness. **We will examine whether this is worth implementing and what disadvantages it may offer.**

In [1]:
# Imports and Setup
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

# Seed for reproducibility
np.random.seed(42)

In [2]:
# Generate a synthetic dataset
X, y = make_classification(n_samples=750, n_features=5, 
                           n_informative=2, n_redundant=0,
                           random_state=42)

# Dynamic Weighted KNN

In [3]:
class DynamicWeightedKNN:
    def __init__(self, k=3, learning_rate=0.01):
        """Initialize the DynamicWeightedKNN with basic parameters."""
        self.k = k  # Number of neighbors to consider
        self.learning_rate = learning_rate  # Learning rate for weight updates
        self.X_train = None  # Training features
        self.y_train = None  # Training labels
        self.weights = None  # Weights for the features
    
    def fit(self, X, y, weights=None, n_iterations=25):
        """Fit the model using the provided training dataset and optional initial weights."""
        self.X_train = X
        self.y_train = y
        self.weights = np.ones(X.shape[1]) if weights is None else weights
        
        for _ in range(n_iterations):
            self.update_weights()
    
    def predict_proba(self, X):
        """Predict the probabilities for the dataset X using the weighted Euclidean distance."""
        probas = []
        for x in X:
            distances = np.array([self.weighted_euclidean_distance(x, x_train) for x_train in self.X_train])
            k_indices = np.argsort(distances)[:self.k]
            k_nearest_labels = self.y_train[k_indices]
            prob = np.mean(k_nearest_labels)
            probas.append(prob)
        return np.array(probas)
    
    def update_weights(self):
        """Update weights using the gradient of the loss function incorporating logistic derivative."""
        y_pred = self.predict_proba(self.X_train)
        error = y_pred - self.y_train
        for i in range(len(self.weights)):
            # Incorporating the logistic derivative for the gradient calculation
            gradient = np.dot(error * y_pred * (1 - y_pred), self.X_train[:, i])
            self.weights[i] -= self.learning_rate * gradient
        self.weights = np.maximum(0, self.weights)  # Ensure weights remain non-negative
    
    def weighted_euclidean_distance(self, x, y):
        """Calculate the weighted Euclidean distance between two vectors."""
        difference = x - y
        weighted_squared_diff = self.weights * np.square(difference)
        return np.sqrt(np.sum(weighted_squared_diff))

In [4]:
class KNN:
    def __init__(self, k=3):
        """Initialize the BasicKNN with the number of neighbors to consider."""
        self.k = k
        self.X_train = None  # Training features
        self.y_train = None  # Training labels
    
    def fit(self, X, y):
        """Fit the model using the provided training dataset."""
        self.X_train = X
        self.y_train = y
    
    def predict(self, X):
        """Predict the labels for the dataset X."""
        predictions = []
        for x in X:
            distances = np.array([self.euclidean_distance(x, x_train) for x_train in self.X_train])
            k_indices = np.argsort(distances)[:self.k]
            k_nearest_labels = self.y_train[k_indices]
            majority_vote = np.argmax(np.bincount(k_nearest_labels))
            predictions.append(majority_vote)
        return np.array(predictions)
    
    def euclidean_distance(self, x, y):
        """Calculate the Euclidean distance between two vectors."""
        return np.sqrt(np.sum((x - y) ** 2))

In [5]:
# Initialize Stratified K-Fold
skf = StratifiedKFold(n_splits=5)

# Prepare lists to store results
standard_knn_accuracies = []
dynamic_weighted_knn_accuracies = []

In [6]:
start_time = time.time()

# Conduct Stratified K-Fold Cross-Validation for Standard KNN
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Standard KNN
    standard_knn = KNN(k=5)
    standard_knn.fit(X_train, y_train)
    standard_predictions = standard_knn.predict(X_test)
    standard_accuracy = accuracy_score(y_test, standard_predictions)
    standard_knn_accuracies.append(standard_accuracy)
    
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time} seconds")
    
# Print average accuracy for Standard KNN
print("Average Accuracy with Standard KNN:", np.mean(standard_knn_accuracies))

Elapsed time: 5.68037223815918 seconds
Average Accuracy with Standard KNN: 0.9333333333333332


In [7]:
start_time = time.time()

# Conduct Stratified K-Fold Cross-Validation for Dynamic Weighted KNN
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Dynamic Weighted KNN
    dynamic_knn = DynamicWeightedKNN(k=5, learning_rate=0.01)
    dynamic_knn.fit(X_train, y_train, n_iterations=25)
    dynamic_predictions = dynamic_knn.predict_proba(X_test) > 0.5  # Thresholding to get binary predictions
    dynamic_accuracy = accuracy_score(y_test, dynamic_predictions)
    dynamic_weighted_knn_accuracies.append(dynamic_accuracy)

elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time} seconds")

# Print average accuracy for Dynamic Weighted KNN
print("Average Accuracy with Dynamic Weighted KNN:", np.mean(dynamic_weighted_knn_accuracies))

Elapsed time: 601.7196817398071 seconds
Average Accuracy with Dynamic Weighted KNN: 0.9306666666666666


## Outcomes

### Performance Analysis

After applying both the Dynamic Weighted KNN and the standard KNN to our selected dataset using stratified cross-validation, we observed key differences in both accuracy and computational efficiency.

**Accuracy**: The Dynamic Weighted KNN showed mixed results in terms of accuracy compared to the standard KNN. While it demonstrated slightly better accuracy on some datasets, it also performed slightly worse on others. This variability suggests that the introduction of feature-specific weights, optimized through gradient descent, might not consistently capture the importance of different features across diverse datasets.

**Speed**: The Dynamic Weighted KNN was significantly slower than the standard KNN. The added computational overhead from updating weights using gradient descent at each training step substantially increases the time required for the model to train. This drawback makes the Dynamic Weighted KNN impractical for scenarios where quick training or real-time prediction is crucial.

### Considerations for Slowness

The slower performance of the Dynamic Weighted KNN is a significant limitation. Even with occasional accuracy improvements, its computational inefficiency could be a dealbreaker in many real-world applications where speed and scalability are paramount.

### Potential Optimizations

Despite its current limitations, there are several strategies that could potentially optimize the performance of the Dynamic Weighted KNN:

- **Efficient Data Structures**: Using more efficient data structures like KD-Trees or Ball Trees could reduce the computational complexity associated with finding the nearest neighbors, thus speeding up both the training and prediction phases.
- **Parallel Processing**: Leveraging parallel processing techniques to distribute the computation of distances and updates across multiple cores or GPUs could significantly reduce execution time.
- **Feature Selection**: Performing feature selection before training to reduce the dimensionality of the data could decrease the number of weights that need to be optimized, thereby simplifying the model and speeding up the calculations.
- **Algorithmic Refinements**: Applying more sophisticated methods for gradient descent (e.g., incorporating momentum or adaptive learning rate techniques) could improve the efficiency of weight updates.

### Conclusion

Given the variable accuracy and significantly slower performance of the Dynamic Weighted KNN, it is not recommended for use over the standard KNN in most practical scenarios. The potential improvements in accuracy do not outweigh the considerable increase in computational costs. However, with targeted optimizations and enhancements, there may be potential to make this model viable for specific applications where the unique advantages of feature-weighted learning can be fully realized.

## Time Complexity

### Training Phase

During the training phase:
- **Distance Calculation**: Calculating the weighted Euclidean distance between all pairs of data points involves O(d) operations per pair. Given n data points, computing distances for all pairs leads to O(n^2 * d) complexity.
- **Weight Updates**: The gradient update for weights based on the calculated distances and errors involves O(n * d) operations per iteration.
- **Iterations**: Repeating the weight update process across t iterations, the total complexity becomes O(t * n^2 * d).